# Workflow Example with ERA5 input convertor
Here we illustrate a complete workflow example using the ERA5 convertor

## Imports

In [ ]:
from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import _non_convertor, ERA5_to_CF

from yaml import safe_load
from typing import Union, List


In [ ]:
#Import Converter - This input converter will not do anything to the data.
ic = vp.InputConverter(ERA5_to_CF)


## 1. Convert the data

### Get the filenames of ERA5 data corresponding to CORDEX variable name


In [ ]:

machine = "hortense"


# User defined variable, here we look at 2m temperature. 
variable = "tas"
dataset = "ERA5"
domain = "belgium" # "belgium"
time_freq = "daily" ## Important to account here are the postprocessed files at daily time resolution and their naming (eg max daily temperature does not correspond to ERA5 variable name as defined!!)

### Generate the path and filename of obs file

In [ ]:
# Get the current file directory and load the CORDEX variables.yml file

# files = Path(__file__).resolve().parent -- this is not working in notebook
src_path = Path("../src/valenspy") ## -- to be removed. 

with open(src_path / "ancilliary_data" / "CORDEX_variables.yml") as file:
    CORDEX_VARIABLES = safe_load(file)

# ERA5-Land has same variable defenitions as ERA5
if dataset == 'ERA5-Land':
    dataset_lookup = 'ERA5'
else: 
    dataset_lookup = dataset

with open(src_path / "ancilliary_data" / Path(dataset_lookup+"_lookup.yml")) as file:
    obs_LOOKUP = safe_load(file)

In [ ]:
# this will be part of the PATH generator. 

# get path of observational datasets from path settings .yml
with open(src_path / "ancilliary_data" / Path("dataset_PATHS.yml")) as file:
    dataset_PATHS = safe_load(file)
directory = dataset_PATHS[machine][dataset]

# get ERA5 variable corresponding to the requested variable using its look-up table
obs_var = obs_LOOKUP[variable]['obs_name']

# get ERA5 subdirectory
subdirectory = Path(directory + '/' + domain +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_long_name']+'/')

# open the EOBS file for the corresponding variable
files = list(subdirectory.glob("*-"+time_freq+"-*"+obs_LOOKUP[variable]['obs_long_name']+"*.nc")) #Select all the netCDF files in the directory

In [ ]:
# test of ERA5 convertor function
ERA5_ds = ic.convert_input(files) # Convert the input to the correct format
ERA5_ds


The file is ValEnsPy CF compliant.
100.00% of the variables are ValEnsPy CF compliant
ValEnsPy CF compliant: ['tas']


<xarray.Dataset> Size: 144MB
Dimensions:    (time: 17520, lat: 25, lon: 41, longitude: 41, latitude: 25)
Coordinates:
  * longitude  (longitude) float32 164B 2.0 2.125 2.25 2.375 ... 6.75 6.875 7.0
  * latitude   (latitude) float32 100B 52.0 51.88 51.75 ... 49.25 49.12 49.0
  * time       (time) datetime64[ns] 140kB 1994-01-01 ... 1995-12-31T23:00:00
  * lon        (lon) float32 164B 2.0 2.125 2.25 2.375 ... 6.625 6.75 6.875 7.0
  * lat        (lat) float32 100B 52.0 51.88 51.75 51.62 ... 49.25 49.12 49.0
Data variables:
    tas        (time, lat, lon) float64 144MB dask.array<chunksize=(8760, 25, 41), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-27 13:21:36 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
## manual implementation ERA5 convertor
file = files

In [ ]:
    # based on the filename in the path, get the name of the dataset
    # Extract the relevant file name
    filename = file[0].stem 

    # Split the directory name by '-' (! era and land are also separated by '-', so order is different from era5)
    filename_parts = filename.split('-')
    
    if filename_parts[1] == 'land': 
        obsdata_name = "ERA5-Land"
    else: 
        obsdata_name = "ERA5"


    # open the observation dataset
    ds = xr.open_mfdataset(file, combine='by_coords', chunks='auto')

    # open observational specific lookup dictionary
    with open(src_path / "ancilliary_data" / Path("ERA5_lookup.yml")) as lookup_file:
        obs_LOOKUP = safe_load(lookup_file)

    # make observation CF compliant
    for var_obs in ds.data_vars: 

        # Get the CORDEX variable in the observational dataset using the observational lookup table  
        var = next((k for k, v in obs_LOOKUP.items() if v.get('obs_name') == var_obs), None)

        if var: # Dont processes variables that are not in the lookup table.

            # update variable name to CORDEX variable name
            ds = ds.rename_vars({obs_LOOKUP[var]["obs_name"]: var})

            # from here on, use CORDEX variable name to access data array and do rest of conversion

            # Unit conversion - hard coded ERA5 units for CORDEX CORE, double check beyond. 
            if (obs_LOOKUP[var]['obs_units'] == 'Celcius') or (obs_LOOKUP[var]['obs_units'] == 'degC'): 
                ds[var] = _convert_Celcius_to_Kelvin(ds[var]) 

            elif obs_LOOKUP[var]['obs_units'] == 'hPa': 
                ds[var] = _convert_hPa_to_Pa(ds[var]) # hPa to Pa

            elif (obs_LOOKUP[var]['obs_units'] == 'mm') or (obs_LOOKUP[var]['obs_units'] == 'mm/hr'): 
                ds[var] = _convert_mm_to_kg_m2s(ds[var]) # mm to kg m^-2 s^-1 conversion function reads time frequency (nseconds) of input ds to do conversion

            elif (obs_LOOKUP[var]['obs_units'] == 'm') or (obs_LOOKUP[var]['obs_units'] == 'm/hr'): 
                ds[var] = _convert_m_to_kg_m2s(ds[var]) # m to kg m^-2 s^-1 conversion function reads time frequency (nseconds) of input ds to do conversion

            elif (obs_LOOKUP[var]['obs_units'] == 'J/m^2'):  
                ds[var] = _convert_m_to_kg_m2s(ds[var]) # m to kg m^-2 s^-1 conversion function reads time frequency (nseconds) of input ds to do conversion_convert_J_m2_to_W_m2

            # update unit attribute
            ds[var].attrs["units"] = CORDEX_VARIABLES[var]["units"] # from the CORDEX look-up table 

            # add necessary metadata
            ds[var].attrs["standard_name"]      = CORDEX_VARIABLES[var]["standard_name"]  # from the CORDEX look-up table
            ds[var].attrs["long_name"]          = CORDEX_VARIABLES[var]["long_name"]  # from the CORDEX look-up table
            ds[var].attrs["original_name"]      = obs_LOOKUP[var]["obs_name"]
            ds[var].attrs["original_long_name"] = obs_LOOKUP[var]["obs_long_name"]

            # rename dimensions
            ds[var] = ds[var].rename({"latitude": "lat", "longitude": "lon"})

            # convert the time dimension to a pandas datetime index --  do we want this to happen within the convertor? Or do we leave it up to the user? 
            ds[var]['time'] = pd.to_datetime(ds[var].time)


            # additional attributes --         
            if obsdata_name == "ERA5": 

                # Extract the relevant directory name
                filename = files[0].stem 

                # Split the directory name by '-' (! era and land are also separated by '-', so order is different from era5)
                parts = filename.split('-')

                # Extract relevant parts assuming freq and domain are always after the first dash and second dash respectively
                ds[var].attrs["freq"]               = filename_parts[1] if len(filename_parts) > 1 else None  # read from file name 
                ds[var].attrs["domain"]             = filename_parts[2] if len(filename_parts) > 2 else None
                ds[var].attrs["dataset"]            = obsdata_name

                # hard coded spatial resolution
                ds[var].attrs["spatial_resolution"] = "0.25deg"  
            
            elif obsdata_name == "ERA5-Land": 

                # Extract relevant parts assuming freq and domain are always after the second dash and third dash respectively
                ds[var].attrs["freq"]               = filename_parts[2] if len(filename_parts) > 2 else None  # read from file name 
                ds[var].attrs["domain"]             = filename_parts[3] if len(filename_parts) > 3 else None

                # hard coded spatial resolution
                ds[var].attrs["spatial_resolution"] = "0.1deg"  
                ds[var].attrs["dataset"]            = obsdata_name


In [ ]:
ds[var]

<xarray.DataArray 'tas' (time: 17520, lat: 25, lon: 41)> Size: 144MB
dask.array<concatenate, shape=(17520, 25, 41), dtype=float64, chunksize=(8760, 25, 41), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 140kB 1994-01-01 ... 1995-12-31T23:00:00
  * lon      (lon) float32 164B 2.0 2.125 2.25 2.375 ... 6.625 6.75 6.875 7.0
  * lat      (lat) float32 100B 52.0 51.88 51.75 51.62 ... 49.25 49.12 49.0
Attributes:
    units:               K
    long_name:           Near-Surface Air Temperature
    standard_name:       air_temperature
    original_name:       t2m
    original_long_name:  2m_temperature
    freq:                daily
    domain:              belgium
    spatial_resolution:  0.25deg